In [2]:
import numpy as np
import scipy 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import allel
import seaborn as sns
import sys
import os
sns.set_style("white")
sns.set_style("ticks")
sns.set_context("notebook")

import zarr
import numcodecs
print("zarr", zarr.__version__, "numcodecs", numcodecs.__version__)
import watermark

zarr 2.4.0 numcodecs 0.6.4


In [48]:
if os.path.exists(zarr_path) & os.path.exists(vcf_path):
    print ("It exists")
else:
    print ("It doesn't exist")
    

It exists


In [158]:
def calculateHaplotypeStatistics(popname):
    print("Now processing {}".format(popname))
    for i in range(1,6):
        print("Now Processing Chromosome {}".format(i))
        population = str(popname)

        chrom = str(i)

        
        #########
        zarr_path = ("merged.bi.lmiss90.ed.chr"+chrom+".vcf.gz.zarr/")#modify according to vcf file
        if os.path.exists(zarr_path) & os.path.isdir(zarr_path):
            print("Zarr already exists")
        else:
            zarr_path = ("merged.bi.lmiss90.ed.chr"+chrom+".vcf.gz.zarr/")
            #herbs_nAm.filtered_snps_final.PASS.bi.QDGt20.MQGt55.SORLt2.depthLt30x.Chr5.HaplotypeData.USA.vcf.gz
            vcf_path = ("merged.bi.lmiss90.ed.chr"+chrom+".vcf.gz") #modify acording to vcf file
            allel.vcf_to_zarr(vcf_path, zarr_path, fields = "*", overwrite = True, log = sys.stdout, group = chrom, region = chrom)
        
        
        #########
        callset = zarr.open_group(zarr_path , mode="r")
        gt_zarr = allel.GenotypeDaskArray(callset[chrom+'/calldata/GT'])
        pos = callset[chrom]['variants/POS'][:]
        samples_meta_file = "merged.bi.lmiss90.ed.allChr.bgl.vcf.indPops.txt"
        colnames = ['ind', 'pop']
        samples = pd.read_csv(samples_meta_file, sep ="\t", header=None, names = colnames)
        print("There are :{} individuals" .format(samples.groupby('pop').size()[population]))
        loc_pop_samples = samples[samples['pop']== population].index.values

        gt_pop = gt_zarr.take(loc_pop_samples, axis=1)
        ac_pop = gt_pop.count_alleles(max_allele=1).compute()
        loc_pop_seg_variants = ac_pop.is_segregating() & ac_pop.is_biallelic_01()
        ac_pop_seg = ac_pop.compress(loc_pop_seg_variants, axis = 0)
        gt_pop_seg = gt_pop.compress(loc_pop_seg_variants, axis =0)

        h_pop_seg = gt_pop_seg.to_haplotypes().compute()
        pos_pop_seg = pos.compress(loc_pop_seg_variants, axis = 0)
        np.count_nonzero(np.diff(pos_pop_seg==0))

        #### nSL ##################     
        nsl_pop_raw = allel.nsl(h_pop_seg,)
        nsl_pop_std, _ = allel.standardize_by_allele_count(nsl_pop_raw, ac_pop_seg[:,1])
        dataset = pandas.DataFrame({'pos': pos_pop_seg[~np.isnan(nsl_pop_std)], 'nsl':np.abs(nsl_pop_std[~np.isnan(nsl_pop_std)])}, columns = ['pos', 'nsl'])
        dataset=dataset.round(4)
        dataset.to_csv(population +"_chr"+chrom+"_stdNsl.txt",sep="\t", index = False)

        #### Hap div ##############
        hap_div=allel.moving_haplotype_diversity(h_pop_seg, 100, start=0, 
                                     stop=None,
                                     step=None)
        hap_div_pos =pos_pop_seg[0::100][0:hap_div.shape[0]]

        dataset = pandas.DataFrame({'pos': hap_div_pos, 'hap_div':hap_div}, columns = ['pos', 'hap_div'])
        dataset=dataset.round(4)
        dataset.to_csv(population +"_chr"+chrom+"_hapDiv.txt",sep="\t", index = False)

        #### Garud H Statistics ######
        garudH = allel.moving_garud_h(h_pop_seg, 500, start=0, stop=None, step=10)
        garudH_Positions = (pos_pop_seg[0::10][0:garudH[3].shape[0]])
        
        garudH_chromosome = np.repeat(i, garudH_Positions.shape[0])
        dataset = pandas.DataFrame({'CHROM': garudH_chromosome, 'START': garudH_Positions, 'H1':garudH[0], 'H12': garudH[1], 'H2_H1': garudH[3]}, columns = ['CHROM','START', 'H1', 'H12', 'H2_H1'])
        dataset=dataset.round(4)
        dataset.to_csv(population +"_chr"+chrom+"_GarudStats.txt",sep="\t", index = False)
    
    print("{} done".format(popname))   

In [ ]:
poplist = ["MISJ", "OHML", "OHPR", "INRC", "NJSC"]
for pop in poplist:
    calculateHaplotypeStatistics(pop)